### Note:  to start Jupyter Notebook with Spark, use the following command:

PYSPARK_DRIVER_PYTHON="jupyter" PYSPARK_DRIVER_PYTHON_OPTS="notebook --ip localhost --port 7777 --no-browser" pyspark --packages org.diana-hep:spark-root_2.11:0.1.11,org.diana-hep:histogrammar-bokeh_2.11:1.0.3,org.diana-hep:histogrammar-sparksql_2.11:1.0.3

# Root file to Spark Dataframe

Below is the code to read in the root file into a Spark DataFrame.

The show() method can show some of the data in table format

The printSchema() can show the tree structure of the data

count() will display the number of rows

In [60]:
#import time
#start_time = time.time()

In [1]:
sc.defaultParallelism

16

In [2]:
#df = sqlContext.read.format("org.dianahep.sparkroot").load("file:/home/ethan/Documents/JL_Summer_2017/Root/data_05_53591_53591_03.root")

In [3]:
#add = sqlContext.read.format("org.dianahep.sparkroot").load("file:/home/ethan/Documents/JL_Summer_2017/Root/data_05_53094_53094_01.root")

In [4]:
#df = df.union(add)
#df.count()

In [5]:
#add.count()

# Importing Packages

In [2]:
from math import *
import numpy as np
from pyspark.sql import Row
import matplotlib.pyplot as plt
#import histogrammar as hist
from pyspark.sql.types import ArrayType,FloatType
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
#import ROOT

In [3]:
#data = sqlContext.read.format("org.dianahep.sparkroot").load('s3://egbuckuseast2/Root_data/
#data_05_53094_53094_01.root')

#from pyspark import SparkContext
#sc = SparkContext()
#from pyspark.sql import SQLContext
#sqlContext = SQLContext(sc)

count = 0
fileList = ["data_05_53094_53094_01.root","data_05_53096_53096_01.root","data_05_53096_53096_02.root",\
            "data_05_53591_53591_03.root","data_05_53137_53137_01.root","data_05_53139_53139_01.root",\
            "data_05_53139_53139_02.root","data_05_53140_53140_01.root","data_05_53140_53140_02.root"]
for filename in fileList:
    print filename
    temp = sqlContext.read.format("org.dianahep.sparkroot").load('s3://egbuckuseast2/Root_data/'+filename)
    if count != 0:
        data = data.union(temp)
    else:
        data = temp
    count+=1

data_05_53094_53094_01.root
data_05_53096_53096_01.root
data_05_53096_53096_02.root
data_05_53591_53591_03.root
data_05_53137_53137_01.root
data_05_53139_53139_01.root
data_05_53139_53139_02.root
data_05_53140_53140_01.root
data_05_53140_53140_02.root


In [4]:
print len(data.columns)
print data.count()
#data.printSchema()

1539
434548


In [5]:
def cut_rows(sincand,vtxPrim,remc,siprob,siecorr,calEmpz,numTracks,Et1ir,Ntrkvtx,Zvtx):
    if sincand != 0:
        if sum(vtxPrim) == 1:
            if remc[3] > 2:
                if len(siprob) > 0:
                    if max(siprob) > 0.9:
                        if len(siecorr) > 0:
                            try:
                                if siecorr[2] > 10:
                                    if calEmpz > 45.0 and calEmpz < 65.0:
                                        if numTracks > 2.0 and numTracks < 5.0:
                                            if Et1ir < 5.0 and Ntrkvtx > 2.0 and abs(Zvtx) < 50.0:
                                                return True
                            except IndexError:
                                return False
    return False

In [6]:
filter_udf = udf(cut_rows, BooleanType())
sample = data.sample(False,0.99999999999999999999999999999999999)
print sample.count()
df_filtered = sample.filter(filter_udf(sample.Sincand,sample.Vtx_prim,sample.Remc,sample.Siprob,sample.Siecorr,\
                                      sample.Cal_empz,sample.Trk_ntracks,sample.Et1ir,\
                                      sample.Ntrkvtx,sample.Zvtx))
print len(df_filtered.columns)

434548
1539


In [7]:
df_filtered.count()

32

In [9]:
MassK=0.493677

panda_df = df_filtered.select("Trk_ntracks","Trk_prim_vtx","Trk_id","Trk_layouter","Sitrknr","Trk_charge","Trk_px",\
                              "Trk_py","Trk_pz").toPandas()

In [15]:
# Testing

count = 0
for row in panda_df.itertuples():
    #do stuff in here
    if count < 3:
        print row
        print row.Index
        print row.Trk_ntracks
        print type(row.Trk_px)
        print row.Trk_px
        print
    count += 1

Pandas(Index=0, Trk_ntracks=3, Trk_prim_vtx=[1, 1, 1], Trk_id=[1, 2, 3], Trk_layouter=[9, 5, 1], Sitrknr=[0, 0, 0], Trk_charge=[1.0, 1.0, -1.0], Trk_px=[0.1881178766489029, 0.49489352107048035, -0.26843389868736267], Trk_py=[-0.2920973300933838, 0.08585832267999649, -0.23683865368366241], Trk_pz=[0.09550194442272186, 0.736717939376831, 0.8279920816421509])
0
3
<type 'list'>
[0.1881178766489029, 0.49489352107048035, -0.26843389868736267]

Pandas(Index=1, Trk_ntracks=3, Trk_prim_vtx=[1, 1, 1], Trk_id=[1, 2, 3], Trk_layouter=[3, 1, 0], Sitrknr=[0, 0, 2], Trk_charge=[1.0, 1.0, -1.0], Trk_px=[0.3039594888687134, -0.32673001289367676, -0.11125334352254868], Trk_py=[0.5909336805343628, 0.5314763188362122, -0.6233611702919006], Trk_pz=[1.9333715438842773, 2.2816150188446045, 3.68863582611084])
1
3
<type 'list'>
[0.3039594888687134, -0.32673001289367676, -0.11125334352254868]

Pandas(Index=2, Trk_ntracks=3, Trk_prim_vtx=[1, 1, 1], Trk_id=[1, 2, 3], Trk_layouter=[1, 1, 1], Sitrknr=[1, 0, 0], Trk

In [ ]:
InvMass = []
for event1 in panda_df.itertuples():
    # do some filtering
    index2 = event1.Index + 1
    while index2 < len(panda_df):  #while the index is less than the number of rows in DF
        # do more filtering
        invMass = calculation here
        InvMass.append(invMass)
        index2 += 1

In [12]:
df_filtered.printSchema()

root
 |-- Runnr: integer (nullable = true)
 |-- Eventnr: integer (nullable = true)
 |-- Date1: integer (nullable = true)
 |-- Date2: integer (nullable = true)
 |-- Weight: float (nullable = true)
 |-- Evtake: integer (nullable = true)
 |-- Evtake_iwant: integer (nullable = true)
 |-- Mvdtake: integer (nullable = true)
 |-- Stttake: integer (nullable = true)
 |-- Tpoltake: integer (nullable = true)
 |-- Lpoltake: integer (nullable = true)
 |-- Fmutake: integer (nullable = true)
 |-- Mbtake: integer (nullable = true)
 |-- Tag6take: integer (nullable = true)
 |-- Itrig: integer (nullable = true)
 |-- Fltw: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- Fltpsw: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- Fltfl: integer (nullable = true)
 |-- Gslt_global: integer (nullable = true)
 |-- Sltw: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- Sltupw: array (nullable = true)
 |    |-- element: integer 

In [18]:
#def q2_filter(siecorr,sith):
#    try:
#        q2 = 4.0*27.5*siecorr[2]*((cos(sith)/2.0)**2.0)
#        if q2 < 2.0 and q2 > 70.0:
#            return False
#        else:
#            return True
#    except IndexError:
#       return False

def q2_filter(q2Array):
    try:
        if min(q2Array)>2:
            if max(q2Array)<70:
                return True
        return False
    except ValueError:
        return False
q2_udf = udf(q2_filter,BooleanType())
#df_filter2 = df_filtered.filter(q2_udf(df_filtered.Siecorr,df_filtered.Sith))
df_filter2 = df_filtered.filter(q2_udf(df_filtered.Siq2el))
df_filter2.count()    

0

In [19]:
df_filtered.select("Siq2el").show()

+--------------------+
|              Siq2el|
+--------------------+
|[34.10392, 1403.2...|
|[1086.5397, 739.3...|
|[1.8346361, 7.205...|
|[9.691471, 8.1584...|
|[21.342632, 311.1...|
|[265.93457, 176.8...|
|[1980.0089, 9.679...|
|[24.223696, 3.099...|
|[1.4151969, 7.207...|
+--------------------+



In [4]:
# data.foreach(function)
# data.foreachPartition(function)
# data.filter()
# udf

selection = data.filter(data.Sincand != 0)
print data.count()
print selection.count()
data.select("Vtx_prim").show()

182606
95946
+--------------------+
|            Vtx_prim|
+--------------------+
|                  []|
|                 [1]|
|                 [1]|
|              [1, 0]|
|              [1, 0]|
|                  []|
|                 [1]|
|                 [1]|
|              [1, 0]|
|     [1, 0, 0, 0, 0]|
|                 [1]|
|        [1, 0, 0, 0]|
|[1, 0, 0, 0, 0, 0...|
|                  []|
|                  []|
|                 [1]|
|           [1, 0, 0]|
|[1, 0, 0, 0, 0, 0...|
|                  []|
|              [1, 0]|
+--------------------+
only showing top 20 rows



In [5]:
def add_to_one(array):
    if sum(array)==1:
        return True
    return False
filter_udf = udf(add_to_one, BooleanType())
df_filtered = selection.filter(filter_udf(selection.Vtx_prim))
df_filtered.count()

76957

In [19]:
df_filtered.Remc

Column<Remc>

In [31]:
def rcal_energy(array):
    #return array[3]
    if array[3] > 2:
        return True
    return False
rcal_udf = udf(rcal_energy,BooleanType())
df_filtered.select("Remc").show()

#test = rcal_udf(df_filtered.Remc)
#print type(test)
#plz = data.withColumn("Test",test)
#plz.select("Test").show(5)

new = data.filter(rcal_udf(data.Remc))
new.count()

+--------------------+
|                Remc|
+--------------------+
|[-2.015098, 0.239...|
|[-1.1364275, 0.45...|
|[0.0666811, -0.04...|
|[-0.37364623, 0.3...|
|[-0.5059691, 1.14...|
|[0.50368035, -0.6...|
|[2.5757477, -1.14...|
|[-9.287206, -7.93...|
|[0.0, 0.0, 0.0, 0...|
|[0.25755203, 0.16...|
|[-3.6059341, 0.75...|
|[0.093001455, -0....|
|[2.6694465, -0.22...|
|[3.023202, 0.8400...|
|[-1.6509546, 1.25...|
|[2.517955, -1.681...|
|[2.9523737, -0.55...|
|[6.227952, 8.5111...|
|[-0.17670819, 0.2...|
|[0.020294249, -0....|
+--------------------+
only showing top 20 rows



121257

In [36]:
def Siprob_filter(array):
    if len(array) > 0:
        if max(array) > 0.9:
            return True
    return False

Siprob_udf = udf(Siprob_filter,BooleanType())
data.filter(Siprob_udf(data.Siprob)).count()

70183

In [40]:
# Need to Check!!  May be filtering too much here with wrong computation
data.select("Siecorr")
def Siecorr_filter(array):
    if len(array) > 0:
        try:
            if array[2] > 10:
                return True
        except IndexError:
                return False
    return False
Siecorr_udf = udf(Siecorr_filter,BooleanType())
data.filter(Siecorr_udf(data.Siecorr)).count()

1224

In [64]:
close = data.filter(rcal_udf(data.Remc) & Siprob_udf(data.Siprob) & Siecorr_udf(data.Siecorr))\
.filter(filter_udf(selection.Vtx_prim))

print close.count()
print close.select("Sincand")
close.select("Sincand").show(5)

933
DataFrame[Sincand: int]


Py4JJavaError: An error occurred while calling o1399.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 111.0 failed 1 times, most recent failure: Lost task 2.0 in stage 111.0 (TID 1302, localhost, executor driver): scala.MatchError: 0 (of class java.lang.Integer)
	at org.apache.spark.sql.catalyst.CatalystTypeConverters$ArrayConverter.toCatalystImpl(CatalystTypeConverters.scala:160)
	at org.apache.spark.sql.catalyst.CatalystTypeConverters$ArrayConverter.toCatalystImpl(CatalystTypeConverters.scala:154)
	at org.apache.spark.sql.catalyst.CatalystTypeConverters$CatalystTypeConverter.toCatalyst(CatalystTypeConverters.scala:103)
	at org.apache.spark.sql.catalyst.CatalystTypeConverters$$anonfun$createToCatalystConverter$2.apply(CatalystTypeConverters.scala:383)
	at org.apache.spark.sql.execution.RDDConversions$$anonfun$rowToRowRdd$1$$anonfun$apply$3.apply(ExistingRDD.scala:60)
	at org.apache.spark.sql.execution.RDDConversions$$anonfun$rowToRowRdd$

In [62]:
work = close.filter(close.Sincand != 0)
work.count()

Py4JJavaError: An error occurred while calling o1255.count.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 103.0 failed 1 times, most recent failure: Lost task 7.0 in stage 103.0 (TID 1205, localhost, executor driver): scala.MatchError: 0 (of class java.lang.Integer)
	at org.apache.spark.sql.catalyst.CatalystTypeConverters$ArrayConverter.toCatalystImpl(CatalystTypeConverters.scala:160)
	at org.apache.spark.sql.catalyst.CatalystTypeConverters$ArrayConverter.toCatalystImpl(CatalystTypeConverters.scala:154)
	at org.apache.spark.sql.catalyst.CatalystTypeConverters$CatalystTypeConverter.toCatalyst(CatalystTypeConverters.scala:103)
	at org.apache.spark.sql.catalyst.CatalystTypeConverters$$anonfun$createToCatalystConverter$2.apply(CatalystTypeConverters.scala:383)
	at org.apache.spark.sql.execution.RDDConversions$$anonfun$rowToRowRdd$1$$anonfun$apply$3.apply(ExistingRDD.scala:60)
	at org.apache.spark.sql.execution.RDDConversions$$anonfun$rowToRowRdd$1$$an

In [54]:
data.select('Sincand').distinct().show()

+-------+
|Sincand|
+-------+
|      1|
|      3|
|      4|
|      2|
+-------+



95946

In [57]:
close.select('Sincand').distinct().show()

Py4JJavaError: An error occurred while calling o1133.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 96.0 failed 1 times, most recent failure: Lost task 7.0 in stage 96.0 (TID 1118, localhost, executor driver): scala.MatchError: 0 (of class java.lang.Integer)
	at org.apache.spark.sql.catalyst.CatalystTypeConverters$ArrayConverter.toCatalystImpl(CatalystTypeConverters.scala:160)
	at org.apache.spark.sql.catalyst.CatalystTypeConverters$ArrayConverter.toCatalystImpl(CatalystTypeConverters.scala:154)
	at org.apache.spark.sql.catalyst.CatalystTypeConverters$CatalystTypeConverter.toCatalyst(CatalystTypeConverters.scala:103)
	at org.apache.spark.sql.catalyst.CatalystTypeConverters$$anonfun$createToCatalystConverter$2.apply(CatalystTypeConverters.scala:383)
	at org.apache.spark.sql.execution.RDDConversions$$anonfun$rowToRowRdd$1$$anonfun$apply$3.apply(ExistingRDD.scala:60)
	at org.apache.spark.sql.execution.RDDConversions$$anonfun$rowToRowRdd$1$

In [71]:
#sincand_filter = data.filter(data.Sincand != 0)
#print sincand_filter.count()
#filtered = sincand_filter.filter(rcal_udf(data.Remc) & Siprob_udf(data.Siprob) & Siecorr_udf(data.Siecorr))\
#.filter(filter_udf(selection.Vtx_prim))

# Runtime Calculation

In [59]:
#print "--- %s seconds ---" % (time.time() - start_time)